# Capstone Project - The Battle of the Neighborhoods
### Buying an apartment in Hamburg, Germany

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project I will try to solve the problem of finding an optimal housing object/location in Hamburg, Germany.
In particular this project will be aimed at the potential buyers of a housing object in Hamburg, Germany. I will limit this housing object to apartments for sale in Hamburg and the target group would be newlyweds, couples or families with children under 10 years old.

The real estate market in Hamburg is notorious for its soaring price over years. The total area of Hamburg is 755,1 square kilometres.There are 7 districts (Altona, Bergedorf, Eimsbüttel, Harburg, Hamburg-Mitte, Hamburg-Nord, Wandsbek) and 104 smaller boroughs that are allocated to those 7 districts. The definition of Hamburg is restricted strictly to the political defined region and not the greater Hamburg Metropolitan area that includes other German federal states such as Schleswig-Holstein and Lower Saxony (Niedersachen), which are the adjoining states of Hamburg.

Given that there are more than 23,000 streets in Hamburg, I will aim at some certain boroughs, exploring the neighborhoods and using the Foursquare API to analyze the common venues in a particular quarter. To make the sales data of apartments in Hamburg more sophisticated, I will use web scrapping techniques to analyze the all the listing advertisements on ImmobilienScout24.de, which is the biggest real estate online platform in Germany. The goal of this capstone project is to recommend young families or couples find an optimal housing object, in this case an apartment, in Hamburg, Germany.

I will use data science analytics to generate some most promising areas based on those criteria. Advantages of each area will then be clearly expressed so that best possible final locations can be chosen by potential targeted buyers.

## Data <a name="data"></a>

Based on definition of the business problem, factors that will influence the discussion are:

- All the street data and their matching boroughs in Hamburg
- The statistical result of overall living quality in all streets in Hamburg area
- The characteristics of apartments available for sale in Hamburg
- Most common venues in a certain neighborhood/borough in Hamburg

I will use regularly spaced grid of locations, centered around city center, to define the areas

Following data sources will be needed to extract/generate the required information:

- **Official Data** from Hamburg government and Statistics office for Hamburg and Schleswig-Holstein
- Characteristics of appartments for sales in Hamburg (as of June 2019) will be retrieved by using **Beautiful Soup** to complete the webscrapping of immobilienscout24.de
- Using **Pandas** to retrieve the basic neighborhodd data from Wikipedia page.
- Important venues in a certain borough in Hamburg will be obtained by using **Foursquare API**

### Official statistical data from government

#### 1. Living quality for rent index 2017 ####
The city-state government of Hamburg published a list containing all the streets and their grades of living quality for the rent index in 2017. This is the most actual data available from the government. There are only two labels for a street, either the living quality is "gut"(English: good) or "normal" (English: normal). Of course, we Germans love to categorize things, so besides the good or normal comment, there is a score for each street in Hamburg. This score is based on six big categories, like the borough status (land value, citizen structure...,etc), grassy area, population density, you name it. The fine line between a street with "good" or "normal" living quality lies in the score of -0,575. However, the contracted company for conducting this survey admitted that the criteria "centrality" (how close is this street to the city) and "tree population" (the private own trees could not be measured properly...) are not considered.

The complete list in German could be download [here](https://www.hamburg.de/contentblob/10020798/e9ae9a0b62cd2f5b86cdd7f7df48dd74/data/d-wohnlagenverzeichnis-2017.pdf).

#### 2. All the street data and their matching boroughs in Hamburg ####

The federal Statistics office for Hamburg and Schleswig-Holstein (Statistikamt Nord) has plenty of interesting statistics for the two states (Hamburg and Schleswig-Holstein) in the northern Germany. The latest published version online is in January, 2017. The complete list in German could be download [here](https://www.statistik-nord.de/fileadmin/Dokumente/Verzeichnisse/SGV_Hamburg_2011_Siebente_Auflage.pdf).

Finally, I combined the data sheets from 1 and 2 and created a new raw data, which join the two tables on the _street_ name "Strasse" and attached the _borough_ tag "Stadtteil" to the living quality table. I dropped all the streets which are actually bridges (German: bruecken), since there are no scores for such an address. (You may live under the bridge for free without renting it or buying it legally....).

*Note*: 
###### 1. The original file of Data 1 contains a typo "kßlmßstraße" should be "kálmánstraße". I adjusted this entry manually. 
###### 2. Between 2017 and 2019 there are some new streets in Hamburg, mostly in Neugraben-Fischbek and Kirchwerder or around the port area. This is also adjusted manually.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_qual = pd.read_csv('https://raw.githubusercontent.com/bekyl/Coursera_Capstone/master/Strassen_Stadtteil_Wohnlage_HH.csv', sep=';')

print ('Data read into a pandas dataframe!')

Data read into a pandas dataframe!


In [2]:
df_qual.shape

(23163, 7)

In [3]:
df_qual.head()

,Strasse,Hausnr1,Hausnr2,Art,Wohnlage,Wohnlagenkennwert,Stadtteil
0,1. Hafenstrasse,1,-1,U,normal,"-21,282",Heimfeld
1,Aalheitengraben,1,-3,U,gut,"2,06",Volksdorf
2,Aalheitengraben,4,-14,G,gut,"1,767",Volksdorf
3,Aalkrautweg,1,-35,U,gut,"4,675",Sasel
4,Aalkrautweg,2,-58,G,gut,"4,72",Sasel


In [5]:
pd.value_counts(df_qual['Wohnlage'].values, sort=False) ## gut means "good" and normal means "normal" in English

normal    15381
gut        7782
dtype: int64

In [4]:
# Let's remove the streets with the one with "normal" living quality from the dataframe
df_gut=df_qual[df_qual.Wohnlage != 'normal']
df_gut.head()

,Strasse,Hausnr1,Hausnr2,Art,Wohnlage,Wohnlagenkennwert,Stadtteil
1,Aalheitengraben,1,-3,U,gut,"2,06",Volksdorf
2,Aalheitengraben,4,-14,G,gut,"1,767",Volksdorf
3,Aalkrautweg,1,-35,U,gut,"4,675",Sasel
4,Aalkrautweg,2,-58,G,gut,"4,72",Sasel
5,Aalort,1,-13,U,gut,"3,075",Sasel


In [5]:
# Now, I'll count how many "good" streets in a certain quarter "Stadtteil" and list the top five good boroughs

df2=df_gut.groupby(['Stadtteil']).count()
df2.Strasse.sort_values(ascending=False).head()

Stadtteil
Sasel            528
Blankenese       494
Volksdorf        433
Niendorf         419
Poppenbuettel    402
Name: Strasse, dtype: int64

##### To sum it up, according to the governmental statistical data, the best boroughs with the most "good" streets to be dewelled in Hamburg would be **Sasel**, **Blankenese** and **Volksdorf**, regardless of the current housing objects availble for sales.

### Characteristics of appartments for sales in Hamburg (Immobilienscout24)

First of all, I need to give credit to balzer82 on Github. He shared a "immoscraper" on Github, so the data could be easily scrapped from the immobilienscout24's website. (check out his sharing link: https://github.com/balzer82/immoscraper/blob/master/immoscraper.ipynb)

I used the immoscrapper to obtain the data available on immobilienscout24.de for all apartments for sale in Hamburg, Germany.
There are 812 objects for sale and the data is saved as a csv file.

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_immo = pd.read_csv('https://raw.githubusercontent.com/bekyl/Coursera_Capstone/master/2019-06-18-Wohnung-Kauf.csv', sep =";")
                      
print ('Data read into a pandas dataframe!')

Data read into a pandas dataframe!


In [7]:
df_immo.head()

,ID,Miete/Kauf,address,balcony,builtInKitchen,city,floorplan,from,garden,lat,livingSpace,lon,numberOfRooms,postcode,price,privateOffer,quarter,title,url
0,108374061,Kauf,"Sophienterrasse 14, Harvestehude, Hamburg",True,False,Hamburg,True,002.01000718389,False,53.57615,257.04,9.99628,4.0,20149,4401000.0,False,Harvestehude,Herzen höher schlagen lassen: Maisonette-Penth...,https://www.immobilienscout24.de/expose/108374061
1,109383549,Kauf,"Sophienterrasse 14, Harvestehude, Hamburg",True,False,Hamburg,True,002.01000718389,False,53.57615,282.31,9.99628,5.0,20149,4238000.0,False,Harvestehude,Einzig aber nicht artig: Top-Roof im Sophienpa...,https://www.immobilienscout24.de/expose/109383549
2,111371370,Kauf,"Tonndorf, Hamburg",False,True,Hamburg,True,001.150603,False,NaN,60.51,NaN,2.0,22045,181000.0,False,Tonndorf,Ansprechender Wohngenuss!,https://www.immobilienscout24.de/expose/111371370
3,105708431,Kauf,"Niendorf, Hamburg",False,True,Hamburg,True,001.150603,True,NaN,67.06,NaN,2.0,22455,244000.0,False,Niendorf,Interessanter Grundriss in beliebter Lage!,https://www.immobilienscout24.de/expose/105708431
4,106069645,Kauf,"Weidengrasweg 1, Neugraben-Fischbek, Hamburg",True,False,Hamburg,True,001.15347,False,53.47536,106.00,9.84621,3.0,21147,399900.0,False,Neugraben-Fischbek,Bezaubernde 3-Zimer-Eigentumswohnung verteilt ...,https://www.immobilienscout24.de/expose/106069645


In [8]:
df_whg=df_immo.groupby(['quarter']).count() # "quarter" means "Stadtteil" in German
df_whg.ID.sort_values(ascending=False).head()

quarter
Rahlstedt     54
Winterhude    40
Stellingen    28
Eimsbüttel    27
Niendorf      25
Name: ID, dtype: int64

Now we have some basic data about quarters "Stadtteile" in Hamburg. Let's create a data frame for all the quarters and the representing boroughs in Hamburg with their geolocations for the battle of neighborhoods chosen from the top three quarters in the lists above **Sasel,  Blankenese ,  Volksdorf , Rahlstedt, Winterhude and Stellingen** in Hamburg.

In [94]:
## Webscrapping from Wikipedia - Information about quarters and boroughs in Hamburg
# Using the Panda library instead of Beautiful Soup

import numpy as np
import pandas as pd

array = pd.read_html("https://de.wikipedia.org/wiki/Liste_der_Bezirke_und_Stadtteile_Hamburgs")

hh_list = pd.DataFrame(array[1]) ## Only the second table of this wikipedia page is needed

hh_list.head()

,Stadtteil,Ortsteile,Bezirk,Fläche(km²),Einwohner,Bevölkerungsdichte(Einwohner/km²),Koordinaten,Karte
0,Hamburg-Altstadt,NaN,Hamburg-Mitte,NaN,2305.000,960,"53° 33′ 0″ N, 10° 0′ 0″ O",NaN
1,HafenCity,NaN,Hamburg-Mitte,NaN,3627.000,1649,"53° 32′ 28″ N, 10° 0′ 1″ O",NaN
2,Neustadt,NaN,Hamburg-Mitte,NaN,12.719,5530,"53° 33′ 7″ N, 9° 59′ 8″ O",NaN
3,St. Pauli,NaN,Hamburg-Mitte,NaN,22.501,9000,"53° 33′ 25″ N, 9° 57′ 50″ O",NaN
4,St. Georg,NaN,Hamburg-Mitte,NaN,11.055,4606,"53° 33′ 18″ N, 10° 0′ 44″ O",NaN


In [95]:
list(hh_list.columns.values)

['Stadtteil',
 'Ortsteile',
 'Bezirk',
 'Fläche(km²)',
 'Einwohner',
 'Bevölkerungsdichte(Einwohner/km²)',
 'Koordinaten',
 'Karte']

In [96]:
## Let's remove unnessary columns and add the geographical latitudes and altitudes to the Hamburg quarter list
hh_list.drop(['Ortsteile','Fläche(km²)','Karte','Koordinaten'], inplace=True, axis=1)

In [97]:
hh_list.head()

,Stadtteil,Bezirk,Einwohner,Bevölkerungsdichte(Einwohner/km²)
0,Hamburg-Altstadt,Hamburg-Mitte,2305.000,960
1,HafenCity,Hamburg-Mitte,3627.000,1649
2,Neustadt,Hamburg-Mitte,12.719,5530
3,St. Pauli,Hamburg-Mitte,22.501,9000
4,St. Georg,Hamburg-Mitte,11.055,4606


In [101]:
## Change the column names in German to English
hh_list.rename({'Stadtteil': 'Quarter', 'Bezirk': 'Borough','Einwohner': 'Population',
                'Bevölkerungsdichte(Einwohner/km²)': 'Density(Population/km²)'}, 
               axis=1, inplace=True)
hh_list.head()

,Quarter,Borough,Population,Density(Population/km²)
0,Hamburg-Altstadt,Hamburg-Mitte,2305.000,960
1,HafenCity,Hamburg-Mitte,3627.000,1649
2,Neustadt,Hamburg-Mitte,12.719,5530
3,St. Pauli,Hamburg-Mitte,22.501,9000
4,St. Georg,Hamburg-Mitte,11.055,4606


In [105]:
## Select those six finalists mentioned above for the battle of neighborhoods

finalist = ['Sasel', 'Blankenese' , 'Volksdorf' , 'Rahlstedt', 'Winterhude','Stellingen']
Quarter_selected = hh_list.loc[hh_list['Quarter'].isin(finalist)]
Quarter_selected = Quarter_selected.reset_index(drop=True)
Quarter_selected

,Quarter,Borough,Population,Density(Population/km²)
0,Blankenese,Altona,13.491,1752
1,Stellingen,Eimsbüttel,25.753,4440
2,Winterhude,Hamburg-Nord,55.651,7323
3,Sasel,Wandsbek,23.647,2815
4,Volksdorf,Wandsbek,20.694,1784
5,Rahlstedt,Wandsbek,91.703,3447


In [108]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
Quarter_selected['Coordinates'] = Quarter_selected['Quarter'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
Quarter_selected[['Latitude', 'Longitude']] = Quarter_selected['Coordinates'].apply(pd.Series)
Quarter_selected

,Quarter,Borough,Population,Density(Population/km²),Coordinates,Latitude,Longitude
0,Blankenese,Altona,13.491,1752,"(53.5575, 9.80306)",53.557500,9.803060
1,Stellingen,Eimsbüttel,25.753,4440,"(53.5967771, 9.9284099)",53.596777,9.928410
2,Winterhude,Hamburg-Nord,55.651,7323,"(53.5963901, 10.0038317)",53.596390,10.003832
3,Sasel,Wandsbek,23.647,2815,"(53.6522, 10.1169)",53.652200,10.116900
4,Volksdorf,Wandsbek,20.694,1784,"(53.6485554, 10.1647514)",53.648555,10.164751
5,Rahlstedt,Wandsbek,91.703,3447,"(53.6038827, 10.1581547)",53.603883,10.158155


In [77]:
# Let's get Hamburg's Latitude and Longitude 

from geopy.geocoders import Nominatim 
address = 'Hamburg'

geolocator = Nominatim(user_agent="Hamburg_explorer")
location = geolocator.geocode(address)
Hamburg_latitude = location.latitude
Hamburg_longitude = location.longitude
print('The geograpical coordinates of Hamburg, Germany are {}, {}.'.format(Hamburg_latitude, Hamburg_longitude))

The geograpical coordinates of Hamburg, Germany are 53.550341, 10.000654.


In [134]:
import folium
# create map of Six quarters for our battle of neighborhodds using latitude and longitude values
hh_finalist = folium.Map(location=[Hamburg_latitude, Hamburg_longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(Quarter_selected['Latitude'], Quarter_selected['Longitude'], 
                           Quarter_selected['Quarter']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(hh_finalist)  
    
hh_finalist

### Foursquare

Using Foursquare API to get info about venues in each are in Hamburg; this is especially of interest for young families in Germany. In particular finding the numbers and coordinates of public high schools which fall in the 'education' category. Also, I would like to explore the most common venues in certain boroughs in Germany. 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [118]:
# @hidden_cell 
CLIENT_ID = 'KJP3HE521VSWTMRH51CJTTXKEBRESASUYUB1BRSVEEL5A3EY' 
CLIENT_SECRET = 'XDNHWQZNMX0A4VVFJJAG0M4V0RIX2S3LJ11AJ4ZDNACHE4UI' 
VERSION = '20180604' 

In [119]:
address = 'Hamburg, Germany'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

53.550341 10.000654


In [120]:
radius = 1000
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'Dist_Latitude', 
                  'Dist_Longitude', 
                  'Venue', 
                  'Venue_Lat', 
                  'Venue_Long', 
                  'Venue_Category']
    
    return(nearby_venues)

In [126]:
Hamburg_6Q_Venues = getNearbyVenues(names=Quarter_selected['Quarter'],
                                   latitudes=Quarter_selected['Latitude'],
                                   longitudes=Quarter_selected['Longitude']
                                  )

Blankenese
Stellingen
Winterhude
Sasel
Volksdorf
Rahlstedt


In [128]:
print ("Shape of the Venues' Dataframe: ", Hamburg_6Q_Venues .shape)
Hamburg_6Q_Venues .tail(5)                       

Shape of the Venues' Dataframe:  (193, 7)


,District,Dist_Latitude,Dist_Longitude,Venue,Venue_Lat,Venue_Long,Venue_Category
188,Rahlstedt,53.603883,10.158155,Juka Dojo,53.605032,10.154634,Gym / Fitness Center
189,Rahlstedt,53.603883,10.158155,Asia lam,53.602716,10.154492,Vietnamese Restaurant
190,Rahlstedt,53.603883,10.158155,Eis Insel,53.601545,10.153541,Ice Cream Shop
191,Rahlstedt,53.603883,10.158155,Von Allwörden,53.596521,10.153373,Bakery
192,Rahlstedt,53.603883,10.158155,H Brockdorffstraße,53.596467,10.153129,Bus Stop


In [129]:
print (Hamburg_6Q_Venues['Venue_Category'].value_counts())

Café                             13
Supermarket                      10
Bakery                           10
Ice Cream Shop                    6
Bus Stop                          6
Park                              5
Hotel                             5
Asian Restaurant                  4
Sushi Restaurant                  4
Restaurant                        4
German Restaurant                 4
Gym / Fitness Center              4
Drugstore                         4
Gas Station                       4
Italian Restaurant                4
Zoo Exhibit                       3
Fast Food Restaurant              3
Greek Restaurant                  3
Trattoria/Osteria                 3
Vietnamese Restaurant             3
Farmers Market                    3
Seafood Restaurant                3
Pharmacy                          2
Indoor Play Area                  2
Taverna                           2
Tea Room                          2
Bank                              2
Convenience Store           

In [133]:
### Number of Unique Categories in the Dataframe 
print('There are {} unique categories.'.format(len(Hamburg_6Q_Venues['Venue_Category'].unique())))
## Check some of the categories randomly  
print (Hamburg_6Q_Venues[['Venue_Category']][10:15])

There are 94 unique categories.
   Venue_Category
10          Hotel
11          Plaza
12       Tea Room
13    Snack Place
14           Café


#### This will conclude the end of week 1 data collection part

## Methodology <a name="methodology"></a>

## Analysis <a name="analysis"></a>

## Results and Discussion <a name="results"></a>

## Conclusion <a name="conclusion"></a>